# `Astropy` models and fitting
If you need to do least square fitting for data to a model a good place to start is `astropy`'s modeling and fitting code.

## Packages being used
+ `astropy`: for modeling and fitting
+ `matplotlib`: for plotting

## Relevant documentation
+ `astropy`: http://docs.astropy.org/en/stable/modeling/index.html

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
import mpl_style
%matplotlib notebook
plt.style.use(mpl_style.style1)

## 1-D model fitting
For an example lets look at the problem of fitting a 1-D model to a spectral line.  First we need to create some fake data:

In [0]:
x = np.linspace(-5., 5., 200)
y = 3 * np.exp(-0.5 * (x - 1.3)**2 / 0.8**2)
y += np.random.normal(0., 0.2, x.shape)

### A trapezoid model

In [0]:
t_init = models.Trapezoid1D(amplitude=1.0, x_0=0.0, slope=0.5)
fit_t = fitting.LevMarLSQFitter()
t = fit_t(t_init, x, y)
print(t)

### A Gaussian model

In [0]:
g_init = models.Gaussian1D(amplitude=1., mean=0, stddev=1.)
fit_g = fitting.LevMarLSQFitter()
g = fit_g(g_init, x, y)
print(g)

### Plotting the results

In [0]:
plt.figure(1, figsize=(8,5))
plt.plot(x, y, 'o', mfc='none')
plt.plot(x, t(x), label='Trapezoid')
plt.plot(x, g(x), label='Gaussian')
plt.xlabel('Position')
plt.ylabel('Flux')
plt.legend(loc=2)
plt.tight_layout()

## Compound models
Models can also be 'added' together before fitting.  To demonstrate lets make a new dataset made up to two Gaussians.

In [0]:
np.random.seed(42)
g1 = models.Gaussian1D(1, 0, 0.2)
g2 = models.Gaussian1D(2.5, 0.5, 0.1)
x = np.linspace(-1, 1, 200)
y = g1(x) + g2(x) + np.random.normal(0., 0.2, x.shape)
print(x.shape)

### Make the model
The model can be 'added' just like arrays:

In [0]:
gg_init = models.Gaussian1D(1, 0, 0.1) + models.Gaussian1D(2, 0.5, 0.1)
fit_gg = fitting.SLSQPLSQFitter()
gg = fit_gg(gg_init, x, y)
print(gg)

### Plot the result

In [0]:
plt.figure(2, figsize=(8, 5))
plt.plot(x, y, 'o', mfc='none')
plt.plot(x, gg(x), label='2 x Gaussian')
plt.xlabel('Position')
plt.ylabel('Flux')
plt.legend(loc=2)
plt.tight_layout()

## `Astropy`'s models
`Astropy` has a large number of 1- and 2-D models built in.  Check out http://docs.astropy.org/en/stable/modeling/index.html#module-astropy.modeling.functional_models for a full list.  If the model you are looking for is not built in, you can always define your own: http://docs.astropy.org/en/stable/modeling/new.html.

## Limitations
+ Uses OLS (or similar) to maximize an objective function (and all the assumptions about the data that go into this, e.g. Gaussian errors)
+ Cov of fit only returned for some fitters (fond on the `fitter.fit_info()` method)